In [256]:
# import the necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ,LabelEncoder
import pickle

In [257]:
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


PreProcess the Data

In [258]:
#drop un-necessary columns
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [259]:
#encode the Gender
encoder_gender=LabelEncoder()
df['Gender']=encoder_gender.fit_transform(df['Gender'])

df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [260]:
#one hot encoding for Geography
from sklearn.preprocessing import OneHotEncoder
one_hot_encoding_geography=OneHotEncoder()
geo_encoder=one_hot_encoding_geography.fit_transform(df[['Geography']])
geo_encoder

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [261]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [262]:
one_hot_encoding_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [263]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=one_hot_encoding_geography.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [264]:
#Combine one hot encoded data with original data
df=pd.concat([df.drop('Geography',axis=1),geo_encoded_df],axis=1)
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [265]:
# Save the encoders and Scaler
with open('encoder_gender.pkl','wb') as file:
    pickle.dump(encoder_gender,file)
with open('one_hot_encoding_geography.pkl','wb') as file:
    pickle.dump(one_hot_encoding_geography,file)

In [266]:
#diviide the Dataset in Dependent and independent data
x=df.drop('Exited',axis=1)
y=df['Exited']

In [267]:
x

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,0.0,1.0,0.0


In [268]:
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [269]:
#split the data in train test split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [270]:
# Scale these features
scaler=StandardScaler()

x_train=scaler.fit_transform(x_train)
x_test=scaler.fit_transform(x_test)


In [271]:
x_train

array([[-0.34459497, -1.09823226, -0.65674999, ..., -1.00171576,
        -0.57559072,  1.73073215],
       [-0.09518109, -1.09823226, -0.46637979, ..., -1.00171576,
         1.73734559, -0.57779016],
       [-0.94734518,  0.91055421, -0.56156489, ...,  0.99828718,
        -0.57559072, -0.57779016],
       ...,
       [ 0.86090545, -1.09823226, -0.08563939, ...,  0.99828718,
        -0.57559072, -0.57779016],
       [ 0.15423279,  0.91055421,  0.39028611, ...,  0.99828718,
        -0.57559072, -0.57779016],
       [ 0.46600014,  0.91055421,  1.1517669 , ..., -1.00171576,
         1.73734559, -0.57779016]])

In [272]:
x_test

array([[-0.52014175,  0.91678285, -0.66770996, ..., -1.00534763,
         1.70626577, -0.56451866],
       [-0.24328152,  0.91678285,  0.38556101, ...,  0.99468081,
        -0.58607517, -0.56451866],
       [-0.46887134, -1.09077084,  0.48131292, ..., -1.00534763,
        -0.58607517,  1.77142063],
       ...,
       [-0.38683868,  0.91678285, -0.66770996, ...,  0.99468081,
        -0.58607517, -0.56451866],
       [-0.68420707,  0.91678285, -0.76346187, ...,  0.99468081,
        -0.58607517, -0.56451866],
       [-1.16614895, -1.09077084, -1.62522903, ..., -1.00534763,
         1.70626577, -0.56451866]])

In [273]:
# save the scaler 
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)


## ANN Implementation

In [274]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [275]:
#Build our ANN model
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),    #HL1 Connected with Unput layer
    Dense(32,activation='relu'),     #HL2
    Dense(1,activation='sigmoid')     #Output Layer
    
])


c:\Users\Syed Ali Zar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [276]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [277]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)

In [278]:
## Compile the Model
model.compile(optimizer=opt , loss='binary_crossentropy',metrics=['accuracy'])

In [279]:
# Set up the TensorBoard

log_dir="logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [280]:
## Set up Early Stopping
early_stopping_callbacks=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [281]:
## Training the model
history=model.fit(
    x_train,y_train,validation_data=[x_test,y_test],epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callbacks]
)

Epoch 1/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.8134 - loss: 0.4475 - val_accuracy: 0.8603 - val_loss: 0.3602
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8587 - loss: 0.3513 - val_accuracy: 0.8620 - val_loss: 0.3305
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8503 - loss: 0.3564 - val_accuracy: 0.8633 - val_loss: 0.3367
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8600 - loss: 0.3404 - val_accuracy: 0.8653 - val_loss: 0.3330
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8530 - loss: 0.3492 - val_accuracy: 0.8617 - val_loss: 0.3375
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8575 - loss: 0.3380 - val_accuracy: 0.8613 - val_loss: 0.3321
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8687 - loss: 0.3322 - val_accuracy: 0.8667 - val_loss: 0.3355


In [282]:
model.save('ANN_model.h5')

In [286]:
## Load tensor board extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [287]:
%tensorboard --logdir logs/fit20241226-225107

Reusing TensorBoard on port 6007 (pid 14660), started 0:00:10 ago. (Use '!kill 14660' to kill it.)